For this experiment, 

First, we import the required modules.

In [1]:
import scipy.io as sio
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

We load the previously saved EMG data (in this case, data was recorded with Matlab/Simulink and stored in a .mat file. This is the reason why we use the scipy.io.loadmat method to load the data). We also set the number of recorded gestures (nGestures), the number of EMG channels (nChannels) and the number of times each gesture was repeated (nIterations). We initialize two empty lists that are going to be filled with data later.

In [2]:
emg_data = sio.loadmat('emg_data_5_class_2ch')
nGestures = 5
nChannels = 2
nIterations = 4
emg = []
emg_seg = []

Ojete calor

In [3]:
for m in range(1,nGestures+1):
    for i in range(nIterations):
        for c in range(1,nChannels+1):
            emg.append(emg_data['motion'+str(m)+'_ch'+str(c)][:,i])

Before computing the features of the EMG data that we are going to use to train the classifier, the signals have to be segmented, that is, they have to be divided into chunks with a predefined length. In this case, we have implemented a function that performs a disjoint segmentation.

In [4]:
def segmentation(emg, samples = 150):
    N = samples # number of samples per segment
    S = int(np.floor(emg.shape[0]/N)) # number of segments
    length = 0
    emg_seg = np.zeros((N,S))
    for s in range(S):
        for n in range(length,N+length):
            emg_seg[n-length,s] = emg[n] # 2D matrix with a EMG signal divided in s segments, each one with n samples
        length = length + N
    length = 0
    return emg_seg

In [5]:
for n in range(len(emg)):
    emg_seg.append(segmentation(emg[n]))

In [6]:
def mav(emg_seg):
    mav = np.mean(np.abs(emg_seg))
    return mav
    
def rms(emg_seg):
    rms = np.sqrt(np.mean(np.power(emg_seg,2)))
    return rms
    
def var(emg_seg):
    N = len(emg_seg)
    var = 1.0/(N-1)*np.sum(np.power(emg_seg,2))
#   var[s] = np.var(emg_seg[:,s])
    return var
    
def ssi(emg_seg):
    ssi = np.sum(np.abs(np.power(emg_seg,2)))
    return ssi
    
def zc(emg_seg):
    zc = np.sum(np.diff(np.sign(emg_seg))!=0)
    return zc
    
def wl(emg_seg):
    wl = np.sum(np.abs(np.diff(emg_seg)))
    return wl
    
def ssc(emg_seg):
    N = len(emg_seg)
    ssc = 0
    for n in range(N-1):
        if n>0 and (emg_seg[n]-emg_seg[n-1])*(emg_seg[n]-emg_seg[n+1])>=0.001:
            ssc += 1
    return ssc
    
def wamp(emg_seg):
    N = len(emg_seg)
    wamp = 0
    for n in range(N-1):
        if np.abs(emg_seg[n]-emg_seg[n+1])>50:
            wamp += 1
    return wamp

In [7]:
feature_list = [mav, rms, var, ssi, zc, wl, ssc, wamp]

In [8]:
def features(segment,feature_list):
    features = np.zeros((1,len(segment)*len(feature_list)))
    i = 0
    for feature in feature_list:
        features[0,i] = feature(segment[0])
        features[0,i+1] = feature(segment[1])
        i +=  len(segment)
    return features

In [9]:
nSegments = len(emg_seg[0][0])
nFeatures = len(feature_list)

feature_matrix = np.zeros((nSegments*nIterations*nGestures,nFeatures*nChannels))
n = 0

for i in range(0,len(emg_seg),nChannels):
    for j in range(nSegments):
        feature_matrix[n] = features((emg_seg[i][:,j],emg_seg[i+1][:,j]),feature_list)
        n += 1

In [10]:
def feature_scaling(feature_matrix,target,reductor=None,scaler=None):
    lda = LDA(n_components=2)    
    minmax = MinMaxScaler(feature_range=(-1,1))
    if not reductor:
        reductor = lda.fit(feature_matrix,target)
    feat_lda = reductor.transform(feature_matrix)
    if not scaler:
        scaler = minmax.fit(feat_lda)
    feat_lda_scaled = scaler.transform(feat_lda)
    
    return feat_lda_scaled,reductor,scaler

In [11]:
def gestures(nSamples,nGestures):
    gestures = []
    for m in range(nGestures):
        gestures.append((m*np.ones((nSamples))))
    gestures = np.array(gestures).ravel()
    return gestures

In [12]:
y = gestures(nIterations*nSegments,nGestures)
[X,reductor,scaler] = feature_scaling(feature_matrix, y)

c:\python27\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [13]:
classifier = SVC(kernel='rbf',C=100,gamma=1)
classifier.fit(X, y)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Test the classifier. We take the last iteration of each recorded gesture. We execute the same steps to have the scaled feature matrix (segmentation -> feature calculation -> target matrix generation -> feature scaling)

In [14]:
emg_test = []
emg_seg_test = []

for m in range(1,nGestures+1):
    for c in range(1,nChannels+1):
        emg_test.append(emg_data['motion'+str(m)+'_ch'+str(c)][:,4]) #motion1_ch1_i1, motion1_ch2_i1, motion1_ch1_i2, motion1_ch2_i2

for n in range(len(emg_test)):
    emg_seg_test.append(segmentation(emg_test[n]))

feature_matrix_test = np.zeros((nGestures*1*len(emg_seg_test[0][0]),len(feature_list)*nChannels))
n = 0

for i in range(0,len(emg_seg_test),nChannels):
    for j in range(len(emg_seg_test[0][0])):
        feature_matrix_test[n] = features((emg_seg_test[i][:,j],emg_seg_test[i+1][:,j]),feature_list)
        n = n + 1

nSegments = len(emg_seg_test[0][0])

y_test = gestures(nSegments,nGestures)

[X_test,reductor,scaler] = feature_scaling(feature_matrix_test,y_test,reductor,scaler)

In [15]:
predict = classifier.predict(X)
print("Classification accuracy = %0.2f percent." %(100*classifier.score(X_test,y_test)))

Classification accuracy = 98.75 percent.
